# Building software to automate engineering the architecture of neural networks
This notebook demonstrates how we try to provide automation for the job of the machine learning engineer.
The idea is that the machine learning engineer provides an initial structure of the neural network they want to train
and then starts our software overnight which tries to improve the accuracy of the neural network by applying mutations to the structure of the network. The traits of the successful mutations are distributed to the next generation of neural networks and through the same mechanism that life uses the fitter neural network structures survive. This is called a genetic algorithm.

## Motivation: The what?
This project aims to figure whether genetic algorithms are suitable as automatic neural network improvement tool.

## How do we try to achieve this?
I elected Rust as the programming language to implement a proof of concept of the idea.
This notebook aims to give you an overview of what has been implemented so far and how the software can be used for training a neural network on data.
With the example of the MNIST data set (28x28 pixel images of the numbers 0 to 9) the usage of our machine learning suite shall be demoed.
We use python as the umbrella tool to prepare this notebook for training a neural network on the MNIST data set.

## Structure of the notebook

### Step 1: Preparing the MNIST data set.
The goal of this step is to transform the pixel data and their labels to two files:
- input.csv: all the pixel data is transformed to one floating point value per pixel (0.0 for white 1.0 for black). One line contains the data of one image
- target.csv: the label of the image on the corresponding line in input.csv. It is structured as a vector of 10 values with one one and nine zeros. The index of the one is the number that is displayed on the image.

## Step 2: Install Rust machine learning library
For running this notebook cell you must have Rust installed. It is the easiest to use the dev container, that way the installation is automated for you.

### Step 3: Doing an initial training on the data set
The structure in the file nn_shape.yaml tells the Rust executable train what shape of the neural network to prepare. A training is started by copy pasting the command printed in the cell output to the Ubuntu shell.
You can find more documentation on how the shape file has to be structured in the README of the Rust machine learning repository here 
[Check out my machine learning project in Rust on GitHub](https://github.com/mpohl100/ml_rust)


### Step 4: Evolve the shape of the neural network
The created neural network of step 2 is used to breed an initial generation of neural networks to train with the MNIST data. The neural networks with the best results get to spread the traits of their shapes for the next generation. The algorithm repeats for the number of generations you specify as command line parameter.

## Conclusion
You don't need to completely run the software or wait until the end. The idea is to familiarize you with the project.
If this approach is something that interests you I am happy to get in contact with you.


## Step 1: Preparing the MNIST data set
Running this cell is essential if you want to run the software on your machine. It prepares the MNIST data set and demonstrates how to prepare data for the use with our machine learning suite

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 16x16 (256 pixels)
x_train = x_train[:, ::2, ::2]  # Downsample by skipping every other pixel
x_test = x_test[:, ::2, ::2]

# Normalize pixel values to range [0.0, 1.0] (0.0 for white, 1.0 for black)
x_train = 1 - x_train / 255.0
x_test = 1 - x_test / 255.0

# Flatten images into 256-pixel vectors
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Convert targets to one-hot encoding
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

# Save input data to input.csv
input_df = pd.DataFrame(x_train_flat)
input_df.to_csv("input.csv", index=False, header=False)


# Save target data to target.csv
target_df = pd.DataFrame(y_train_onehot)
target_df.to_csv("target.csv", index=False, header=False)

print("Saved input.csv and target.csv.")

## Step 2: Install Rust machine learning library

In [ ]:
# clone github.com/mpohl100/ml_rust and cargo build --release

import os
import subprocess

def clone_and_build(repo_url, branch="main"):
    """
    Clones a GitHub repository and runs `cargo build --release`.

    Args:
        repo_url (str): The GitHub repository URL to clone.
        branch (str): The branch to checkout. Default is "main".
    """
    try:
        # Extract repository name from URL
        repo_name = repo_url.split("/")[-1].replace(".git", "")
        
        # Check if the repository already exists
        if not os.path.exists(repo_name):
            print(f"Cloning {repo_url} into {os.getcwd()}...")
            subprocess.run(
                ["git", "clone", "--branch", branch, repo_url],
                check=True
            )
        else:
            print(f"Repository {repo_name} already exists. Skipping clone.")
            # do a git pull
            os.chdir(repo_name)
            print("Running `git pull`...")
            subprocess.run(["git", "pull"], check=True)
            os.chdir("..")

        # Change into the repository directory
        repo_path = os.path.join(os.getcwd(), repo_name)
        os.chdir(repo_path)

        # Run `cargo build --release`
        print("Running `cargo build --release`...")
        subprocess.run(["cargo", "build", "--release"], check=True)

        print("Build completed successfully!")

    except subprocess.CalledProcessError as e:
        print(f"An error occurred while executing a command: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Change back to the original directory
        os.chdir("..")

repo_url = "https://github.com/mpohl100/ml_rust.git"
clone_and_build(repo_url)


## Step 3: Doing an initial training on the data set
Running the cell prepares the shell command for you. please copy paste it and run it in your shell.

In [ ]:
# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
shape_file = "nn_shape.yaml"
validation_split = 0.7
learning_rate = 0.05
epochs = 10
tolerance = 0.2
batch_size = 32
use_adam = True

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/train",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--shape-file", shape_file,
    "--validation-split", str(validation_split),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--batch-size", str(batch_size),
    "--use-adam" if use_adam else ""
]

#print the command
print(" ".join(command))

## Step 4: Evolve the shape of the neural network
This cell prepares the command you need to paste to your shell and run in order to do automated neural network improving.
This command is added for demonstration purposes, it will take a couple of hours to finish.

In [ ]:
# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
nb_threads = 4
validation_split = 0.7
learning_rate = 0.001
epochs = 10
tolerance = 0.1
batch_size = 32
use_adam = True
num_generations = 100
log_level = 1
population_size = 4
num_offsprings = 10

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/nn_generator",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--nb-threads", str(nb_threads),
    "--validation-split", str(validation_split),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--batch-size", str(batch_size),
    "--use-adam" if use_adam else "",
    "--num-generations", str(num_generations),
    "--log-level", str(log_level),
    "--population-size", str(population_size),
    "--num-offsprings", str(num_offsprings)
]

#print the command
print(" ".join(command))